# A brief summary of the Standard Library in C++ (98)
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [Containers](#containers)
    * [`vector`](#vector)
<hr>

The C++ standard library resides in the namespace `std`.

* Header files are included as `<header-name>`, omitting the `.h` extension. For example, `<vector>`.

* Header files from the C standard library may be included as `<c-header-name>`. For example, `<cstdio>`.

* Header files from the C standard library may be included directly into the global namespace as `<header-name.h>`. For example, `<stdio.h>`.

* User-defined header files may be included as `"header-name.h"`. For example, `"myvector.h"`.

## Containers <a class="anchor" id="containers"></a>

A *container* is an object that stores a collection of other objects. They are implemented as class templates.

Containers share some standard methods, though with no derivation from a common base. This is to enable inlining of most method calls, while still allowing for the implementation of pieces of code that deal with containers generically, and not a particular type of container, through the use of templates. This comes, ofcourse, at the expense of inflated code size.

### `vector` <a class="anchor" id="vector"></a>

A `vector` is a dynamic array, that is able to grow dynamically and implicitly.

In [ ]:
std::vector<int> vec1;          /* 0 elements */

In [ ]:
std::vector<int> vec2(5);       /* 5 elements, copy-initialized with a default-initialized object */

In [ ]:
std::vector<int> vec3(5, 3);    /* 5 elements, copy-initialized with a passed object */

Growth involves possible re-allocation of elements, similar to how a function like `realloc` operates. Hence, growth should not occur with every append to its content. Therefore, `vector` defines both methods `size`, that returns the number of elements in the `vector`, and `capacity`, that returns the maximum number of elements that a `vector` can currently withstand without incurring a growth overhead.

*Note:* A `vector` never shrinks in capacity.

In [42]:
#include <vector>
#include <iostream>

template<class T>
void summary(std::vector<T> &vec) {
    std::cout << "Size: " << vec.size() << ", Capacity: " << vec.capacity() << '\n';
}

int main() {
    std::vector<int> vec1;
    summary(vec1);
    
    std::vector<int> vec2(5);
    summary(vec2);
}

Size: 0, Capacity: 0
Size: 5, Capacity: 5


`vector` defines the method `reserve`, that accepts the number of elements to set the capacity of the `vector` to. If the argument is less than the current capacity of the `vector`, nothing happens.

*Note:* `reserve` merely allocates memory to avoid a growth overhead later on. It does not construct any objects.

`vector` also defines the method `resize`, that accepts the number of elements to set the size of the `vector` to. The argument may be less than the current size of the `vector`.

*Note:* Unlike `reserve`, `resize` initializes memory with copy-initialized objects, copied from a default-initialized object, or an optionally passed object.

In [48]:
#include <vector>
#include <iostream>

template<class T>
void summary(std::vector<T> &vec) {
    std::cout << "Size: " << vec.size();
    std::cout << ", Capacity: " << vec.capacity();
    std::cout << ", [";
    for (int i = 0; i < vec.size(); i++) {
        std::cout << vec[i] << ", ";
    }
    std::cout << "\b\b]\n";
}

template<class T>
void print_vec(std::vector<T> &vec) {
    for (int i = 0; i < vec.size(); i++) {
        std::cout << vec[i] << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector<int> vec(5, 1);        /* 5 elements, copy-initialized with '1' */
    summary(vec);
    
    vec.reserve(10);                   /* capacity guranteed to be at least '10' */
    summary(vec);
    
    vec.reserve(8);                    /* capacity guranteed to be at least '8' (already '10') */
    summary(vec);
    
    vec.resize(8, 2);                  /* enlarged to 8 elements, last three initialized with '2' */
    summary(vec);
    
    vec.resize(7);                     /* shrinked to 7 elements */
    summary(vec);
}

Size: 5, Capacity: 5, [1, 1, 1, 1, 1, ]
Size: 5, Capacity: 10, [1, 1, 1, 1, 1, ]
Size: 5, Capacity: 10, [1, 1, 1, 1, 1, ]
Size: 8, Capacity: 10, [1, 1, 1, 1, 1, 2, 2, 2, ]
Size: 7, Capacity: 10, [1, 1, 1, 1, 1, 2, 2, ]


`vector` defines the method `max_size`, that returns the maximum number of elements that `vector` can withstand.

In [52]:
#include <iostream>
#include <vector>

int main() {
    std::vector<float> vfloat;
    std::cout << vfloat.max_size() << '\n';
    
    std::vector<double> vdouble;
    std::cout << vdouble.max_size() << '\n';
}

2305843009213693951
1152921504606846975


*Note:* Since, usually, `double` occupies double the number of bytes of `float`, the maximum size of `vector<double>` is half the maximum size of `vector<float>`.

`vector` defines the method `empty`, that evaluates and returns `size() == 0`, as a `bool`.

In [54]:
int main() {
    bool x = false;
}